# Predict Workout

Predict Workout Day and distance

In [ ]:
#!pip install sklearn

In [ ]:
import pandas as pd
from datetime import datetime, timedelta

filepath = "data/"

def loadCleanData():
    df = pd.read_csv(filepath+ 'activities_clean.csv')
    df['Week'] = pd.to_datetime(df.Week)
    df['Activity Date'] = pd.to_datetime(df['Activity Date']) + timedelta(hours=3)
    return df

data = loadCleanData()
data.head()

### Feature Extraction

In [ ]:
data['year'] = list(map(lambda t : t.year, data['Activity Date']))

In [ ]:
data['month'] = list(map(lambda t : t.month, data['Activity Date']))

In [ ]:
data['day'] = list(map(lambda t : t.day, data['Activity Date']))

In [ ]:
data['hour'] = list(map(lambda t : t.hour, data['Activity Date']))

In [ ]:
data['minute'] = list(map(lambda t : t.minute, data['Activity Date']))

In [ ]:
data['dayOfWeek'] = list(map(lambda t : t.dayofweek, data['Activity Date']))

In [ ]:
data['isWeekend'] = ((pd.DatetimeIndex(data['Activity Date']).dayofweek) // 5 == 1).astype(int)

### Transform Data

In [ ]:
# convert string data to numeric value for decision making
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

le.fit(data['Activity Type'])
le.classes_

In [ ]:
vals = le.transform(le.classes_)
vals

In [ ]:
data['rideType'] = le.transform(data['Activity Type'])
data['rideType']

### Train Model

In [ ]:
# Select features as list of array
X = data[['year','month','day','hour','minute','dayOfWeek','isWeekend','rideType']]
X = X.to_numpy()
X

### Predict

In [ ]:
Y = data['Distance']
Y = Y.to_numpy()
Y

## Feature Evaluation

In [ ]:
# ride distribution per hour
data.plot(kind='scatter', x='hour', y='Distance', xticks=data['hour'], figsize=(14,8))

In [ ]:
# ride distribution per hour
data.plot(kind='scatter', x='dayOfWeek', y='Distance', xticks=data['dayOfWeek'], figsize=(14,8))

In [ ]:
# ride distribution per hour
data.plot(kind='scatter', x='isWeekend', y='Distance', xticks=data['isWeekend'], figsize=(14,8))